In [3]:
import os, time, sys
from pathlib import Path

SESSION_NAME = "LegoTest"

ROOT = Path().absolute().parent
RAW_INPUT_PATH = Path(f"{ROOT}/data/1_raw_input")
INPUT_PATH = Path(f"{ROOT}/data/2_input")
RESULTS_PATH = Path(f"{ROOT}/data/3_results")

SESSION_ID = SESSION_NAME if SESSION_NAME else str(int(time.time()))
SESSION_PATH = Path(f"{RESULTS_PATH}/{SESSION_ID}")

!mkdir -p {RAW_INPUT_PATH}
!mkdir -p {INPUT_PATH}
!mkdir -p {SESSION_PATH}

print(f"Created new session with ID {SESSION_ID} under {RESULTS_PATH}")

Created new session with ID LegoTest under /mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/3_results


In [ ]:
##### Data Preparation ( raw input -> input folder)
%cd {ROOT}

raw_input_files = [x for x in RAW_INPUT_PATH.glob('**/*') if x.is_file()]
print(f"found {len(raw_input_files)} raw input files")
_f = "/mnt/c/Users/tworkool/Documents/dev/python/historical-photo-sfm-pipeline/data/1_raw_input/lego.mp4" #str(raw_input_files[0])
print(_f)

!bash scripts/run_ffmpeg.sh {{SESSION_NAME}} {{_f}} 2

In [4]:
import pycolmap

COLMAP_MVS_PATH = Path(f"{SESSION_PATH}/mvs")
COLMAP_DB_PATH = Path(f"{SESSION_PATH}/database.db")

!mkdir -p {COLMAP_MVS_PATH}

pycolmap.extract_features(COLMAP_DB_PATH, INPUT_PATH)
# dense reconstruction
#pycolmap.undistort_images(mvs_path, output_path, image_dir)
#pycolmap.patch_match_stereo(mvs_path)  # requires compilation with CUDA
#pycolmap.stereo_fusion(mvs_path / "dense.ply", mvs_path)

W20240120 15:28:19.759575  1492 feature_extraction.cc:403] Your current options use the maximum number of threads on the machine to extract features. Extracting SIFT features on the CPU can consume a lot of RAM per thread for large images. Consider reducing the maximum image size and/or the first octave or manually limit the number of extraction threads. Ignore this warning, if your machine has sufficient memory for the current settings.
I20240120 15:28:19.760308  1549 misc.cc:198] 
Feature extraction
I20240120 15:28:21.848876  1566 feature_extraction.cc:254] Processed file [1/100]
I20240120 15:28:21.848932  1566 feature_extraction.cc:257]   Name:            000001.jpg
I20240120 15:28:21.848944  1566 feature_extraction.cc:283]   Dimensions:      800 x 800
I20240120 15:28:21.848978  1566 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I20240120 15:28:21.848989  1566 feature_extraction.cc:289]   Focal Length:    960.00px
I20240120 15:28:21.849002  1566 feature_extraction

In [5]:
pycolmap.match_exhaustive(COLMAP_DB_PATH)

I20240120 15:29:06.871918  1575 misc.cc:198] 
Exhaustive feature matching
I20240120 15:29:06.892172  1575 feature_matching.cc:231] Matching block [1/2, 1/2]
I20240120 15:31:27.091264  1575 feature_matching.cc:46]  in 140.199s
I20240120 15:31:27.295648  1575 feature_matching.cc:231] Matching block [1/2, 2/2]
I20240120 15:33:54.129055  1575 feature_matching.cc:46]  in 146.833s
I20240120 15:33:54.198055  1575 feature_matching.cc:231] Matching block [2/2, 1/2]
I20240120 15:35:57.127766  1575 feature_matching.cc:46]  in 122.930s
I20240120 15:35:57.584610  1575 feature_matching.cc:231] Matching block [2/2, 2/2]
I20240120 15:38:05.318575  1575 feature_matching.cc:46]  in 127.734s
I20240120 15:38:05.684470  1575 timer.cc:91] Elapsed time: 8.980 [minutes]


In [6]:
maps = pycolmap.incremental_mapping(COLMAP_DB_PATH, INPUT_PATH, SESSION_PATH)
maps[0].write(SESSION_PATH)

I20240120 15:40:51.952667  1668 misc.cc:198] 
Loading database
I20240120 15:40:52.019912  1668 database_cache.cc:54] Loading cameras...
I20240120 15:40:52.023458  1668 database_cache.cc:64]  100 in 0.004s
I20240120 15:40:52.023502  1668 database_cache.cc:72] Loading matches...
I20240120 15:40:52.107344  1668 database_cache.cc:78]  1903 in 0.084s
I20240120 15:40:52.107399  1668 database_cache.cc:94] Loading images...
I20240120 15:40:52.392769  1668 database_cache.cc:143]  100 in 0.285s (connected 100)
I20240120 15:40:52.392833  1668 database_cache.cc:154] Building correspondence graph...
I20240120 15:40:52.424099  1668 database_cache.cc:190]  in 0.031s (ignored 0)
I20240120 15:40:52.424309  1668 timer.cc:91] Elapsed time: 0.007 [minutes]
I20240120 15:40:52.443645  1668 misc.cc:198] 
Finding good initial image pair
I20240120 15:40:52.568537  1668 misc.cc:198] 
Initializing with image pair #18 and #13
I20240120 15:40:52.569299  1668 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.316432e+01    0.00e+00    4.97e+02   0.00e+00   0.00e+00  1.00e+04        0    9.28e-04    2.69e-03
   1  2.931183e+01    3.85e+00    2.15e+02   2.08e+01   9.69e-01  3.00e+04        1    1.66e-03    4.39e-03
   2  2.898766e+01    3.24e-01    8.30e+01   1.35e+01   9.88e-01  9.00e+04        1    4.23e-04    4.85e-03
   3  2.897201e+01    1.57e-02    5.73e+00   4.92e+00   9.88e-01  2.70e+05        1    3.65e-04    5.24e-03
   4  2.897174e+01    2.71e-04    9.85e-01   5.74e-01   1.00e+00  8.10e+05        1    4.63e-04    5.72e-03
   5  2.897173e+01    4.19e-06    1.91e+00   2.50e-01   9.97e-01  2.43e+06        1    4.59e-04    6.21e-03
   6  2.897173e+01    7.51e-06    1.92e+00   8.38e-01   7.19e-01  2.65e+06        1    5.37e-04    6.76e-03
   7  2.897172e+01    1.01e-05    2.30e+00   9.16e-01   7.07e-01  2.86e+06        1    4.31e-04    7.22e-03
   8  2.897171e+01    1.08e-

I20240120 15:40:52.630386  1668 misc.cc:205] 
Bundle adjustment report
------------------------
I20240120 15:40:52.630446  1668 bundle_adjustment.cc:942] 
    Residuals : 492
   Parameters : 378
   Iterations : 101
         Time : 0.0568569 [s]
 Initial cost : 0.259629 [px]
   Final cost : 0.242657 [px]
  Termination : No convergence

I20240120 15:40:52.630728  1668 incremental_mapper.cc:160] => Filtered observations: 0
I20240120 15:40:52.630738  1668 incremental_mapper.cc:167] => Filtered images: 0
I20240120 15:40:52.651400  1668 misc.cc:198] 
Registering image #14 (3)
I20240120 15:40:52.651441  1668 incremental_mapper.cc:495] => Image sees 95 / 2045 points
I20240120 15:40:52.665648  1668 misc.cc:205] 
Pose refinement report
----------------------
I20240120 15:40:52.665719  1668 bundle_adjustment.cc:942] 
    Residuals : 182
   Parameters : 8
   Iterations : 8
         Time : 0.00319695 [s]
 Initial cost : 0.707431 [px]
   Final cost : 0.40265 [px]
  Termination : Convergence

I202401

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  2.140319e+02    0.00e+00    5.24e+01   0.00e+00   0.00e+00  1.00e+04        0    2.05e-03    5.07e-03
   1  2.133908e+02    6.41e-01    5.74e+00   2.24e+00   9.99e-01  3.00e+04        1    4.55e-03    9.65e-03
   2  2.133690e+02    2.18e-02    1.68e+00   2.24e+00   1.02e+00  9.00e+04        1    4.20e-03    1.39e-02
   3  2.133653e+02    3.65e-03    1.90e+00   1.71e+00   1.00e+00  2.70e+05        1    4.19e-03    1.81e-02
   4  2.133631e+02    2.16e-03    5.72e+00   1.76e+00   9.69e-01  8.10e+05        1    4.19e-03    2.23e-02
   5  2.133615e+02    1.66e-03    1.27e+01   2.20e+00   8.73e-01  1.39e+06        1    4.33e-03    2.67e-02
   6  2.133608e+02    6.66e-04    4.26e+00   1.26e+00   9.61e-01  4.16e+06        1    4.41e-03    3.12e-02
   7  2.133607e+02    8.30e-05    1.94e+00   5.48e-01   9.89e-01  1.25e+07        1    4.21e-03    3.54e-02
   8  2.133607e+02    2.23e-

I20240120 15:40:53.092980  1668 misc.cc:205] 
Bundle adjustment report
------------------------
I20240120 15:40:53.093041  1668 bundle_adjustment.cc:942] 
    Residuals : 7384
   Parameters : 4333
   Iterations : 11
         Time : 0.0926011 [s]
 Initial cost : 0.372431 [px]
   Final cost : 0.320756 [px]
  Termination : Convergence

I20240120 15:40:53.098398  1668 incremental_mapper.cc:78] => Merged observations: 0
I20240120 15:40:53.098456  1668 incremental_mapper.cc:79] => Completed observations: 68
I20240120 15:40:53.098465  1668 incremental_mapper.cc:81] => Filtered observations: 46
I20240120 15:40:53.098470  1668 incremental_mapper.cc:90] => Changed observations: 0.030878
I20240120 15:40:53.212051  1668 misc.cc:205] 
Bundle adjustment report
------------------------
I20240120 15:40:53.212112  1668 bundle_adjustment.cc:942] 
    Residuals : 7422
   Parameters : 4303
   Iterations : 11
         Time : 0.109141 [s]
 Initial cost : 0.291922 [px]
   Final cost : 0.269733 [px]
  Termina

...

I20240120 15:42:28.034747  1668 misc.cc:205] 
Bundle adjustment report
------------------------
I20240120 15:42:28.034814  1668 bundle_adjustment.cc:942] 
    Residuals : 174570
   Parameters : 61081
   Iterations : 13
         Time : 2.67249 [s]
 Initial cost : 1.89678 [px]
   Final cost : 1.36045 [px]
  Termination : Convergence

I20240120 15:42:28.132689  1668 incremental_mapper.cc:175] => Completed observations: 46
I20240120 15:42:28.166633  1668 incremental_mapper.cc:178] => Merged observations: 53
I20240120 15:42:28.189749  1668 incremental_mapper.cc:160] => Filtered observations: 1665
I20240120 15:42:28.189819  1668 incremental_mapper.cc:119] => Changed observations: 0.020210
I20240120 15:42:28.189852  1668 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.969633e+04    0.00e+00    8.92e+04   0.00e+00   0.00e+00  1.00e+04        0    5.06e-02    4.09e-01
   1  3.240198e+04    7.29e+03    1.43e+05   3.54e+02   7.63e-01  1.17e+04        1    3.05e-01    7.13e-01
   2  2.998872e+04    2.41e+03    3.35e+03   7.36e+01   1.00e+00  3.51e+04        1    1.65e-01    8.78e-01
   3  2.997997e+04    8.75e+00    3.10e+02   2.41e+01   1.00e+00  1.05e+05        1    2.02e-01    1.08e+00
   4  2.997945e+04    5.13e-01    2.60e+01   3.91e+00   1.00e+00  3.16e+05        1    2.00e-01    1.28e+00
   5  2.997942e+04    3.83e-02    2.63e+00   8.61e-01   1.01e+00  9.48e+05        1    2.27e-01    1.51e+00
   6  2.997941e+04    6.59e-04    7.58e-01   1.08e-01   1.01e+00  2.84e+06        1    2.33e-01    1.74e+00


I20240120 15:42:30.069594  1668 misc.cc:205] 
Bundle adjustment report
------------------------
I20240120 15:42:30.069676  1668 bundle_adjustment.cc:942] 
    Residuals : 171332
   Parameters : 59998
   Iterations : 7
         Time : 1.7479 [s]
 Initial cost : 0.481344 [px]
   Final cost : 0.418304 [px]
  Termination : Convergence

I20240120 15:42:30.158610  1668 incremental_mapper.cc:175] => Completed observations: 317
I20240120 15:42:30.190073  1668 incremental_mapper.cc:178] => Merged observations: 27
I20240120 15:42:30.210136  1668 incremental_mapper.cc:160] => Filtered observations: 31
I20240120 15:42:30.210211  1668 incremental_mapper.cc:119] => Changed observations: 0.004377
I20240120 15:42:30.210240  1668 misc.cc:198] 
Global bundle adjustment


iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  3.035523e+04    0.00e+00    5.84e+03   0.00e+00   0.00e+00  1.00e+04        0    5.03e-02    4.35e-01
   1  3.009778e+04    2.57e+02    4.82e+02   3.05e+01   9.99e-01  3.00e+04        1    2.95e-01    7.30e-01
   2  3.009698e+04    7.97e-01    7.69e+00   8.31e+00   1.00e+00  9.00e+04        1    1.58e-01    8.88e-01
   3  3.009693e+04    4.90e-02    1.96e+00   1.51e+00   1.00e+00  2.70e+05        1    2.56e-01    1.14e+00
   4  3.009693e+04    5.75e-03    1.54e+00   3.98e-01   1.00e+00  8.10e+05        1    1.57e-01    1.30e+00
   5  3.009693e+04    1.76e-04    2.96e-01   6.83e-02   1.01e+00  2.43e+06        1    1.68e-01    1.47e+00


I20240120 15:42:31.817068  1668 misc.cc:205] 
Bundle adjustment report
------------------------
I20240120 15:42:31.817149  1668 bundle_adjustment.cc:942] 
    Residuals : 171904
   Parameters : 59977
   Iterations : 6
         Time : 1.4753 [s]
 Initial cost : 0.420217 [px]
   Final cost : 0.418425 [px]
  Termination : Convergence

I20240120 15:42:31.897537  1668 incremental_mapper.cc:175] => Completed observations: 22
I20240120 15:42:31.925338  1668 incremental_mapper.cc:178] => Merged observations: 0
I20240120 15:42:31.943205  1668 incremental_mapper.cc:160] => Filtered observations: 5
I20240120 15:42:31.943269  1668 incremental_mapper.cc:119] => Changed observations: 0.000314
I20240120 15:42:31.943370  1668 incremental_mapper.cc:167] => Filtered images: 0
I20240120 15:42:31.969627  1668 timer.cc:91] Elapsed time: 1.667 [minutes]


In [ ]:
camera = pycolmap.Camera(
    model=camera_model_name_or_id,
    width=width,
    height=height,
    params=params,
)

import pycolmap
reconstruction = pycolmap.Reconstruction("path/to/reconstruction/dir")
print(reconstruction.summary())

for image_id, image in reconstruction.images.items():
    print(image_id, image)

for point3D_id, point3D in reconstruction.points3D.items():
    print(point3D_id, point3D)

for camera_id, camera in reconstruction.cameras.items():
    print(camera_id, camera)

reconstruction.write("path/to/reconstruction/dir/")